**Loading All the Library Needed**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import os.path

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

from tensorflow import keras

from sklearn.metrics import r2_score

import csv

# ICA

**Loading the Model**

In [ ]:
from keras.models import model_from_json
json_file = open('/content/drive/MyDrive/Data/Model/VGG16/ICA/model.json', 'r')
model_json = json_file.read()
json_file.close()
# Loading the model
model_ica = model_from_json(model_json)
# Loading the weights
model_ica.load_weights('/content/drive/MyDrive/Data/Model/VGG16/ICA/model.h5')
print('Model is successfully loaded from Drive')
print(model_ica.summary())

Model is successfully loaded from Drive
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 216, 144, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 6, 4, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________
None


Evaluation Looping for Each Version in Source 3

In [ ]:
def evaluate(model, base, i):
  image_dir = Path('/content/drive/MyDrive/Data/CNN {}/source3/v{}'.format(base, i))
  filepaths = pd.Series(list(image_dir.glob(r'**/*.png')), name='Filepath').astype(str)
  bpms = pd.Series(filepaths.apply(lambda x: (os.path.split(os.path.split(x)[1])[1])[:-4]), name='BPMS').astype(np.float64)
  images = pd.concat([filepaths, bpms], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

  # Printing Info
  print("-----Currently Working on Version {}-----".format(i))
  print('Total Data: {}'.format(len(images)))

  # Building Test Generator
  test_image_df = images.sample(len(images), random_state = 1).reset_index(drop=True)
  test_generator = tf.keras.preprocessing.image.ImageDataGenerator()
  test_images_flow = test_generator.flow_from_dataframe(
    dataframe=test_image_df,
    x_col='Filepath',
    y_col='BPMS',
    target_size=(216, 144),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
  )

  predicted_bpm = np.squeeze(model.predict(test_images_flow))
  true_bpm = test_images_flow.labels

  # Getting MAE
  mae = np.mean(np.abs(predicted_bpm-true_bpm))
  print('MAE for Version {}: {}'.format(i, mae))

  # Printing to CSV file
  header = ['true bpm', 'predicted bpm']
  data = []
  for x in range(len(true_bpm)):
    data.append([true_bpm[x], predicted_bpm[x]])
  
  with open('{}_v{}.csv'.format(base, i), 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

In [ ]:
for i in range(1, 8):
  evaluate(model_ica, 'ICA', i)

-----Currently Working on Version 1-----
Total Data: 104
Found 104 validated image filenames.
MAE for Version 1: 8.849217183979922
-----Currently Working on Version 2-----
Total Data: 106
Found 106 validated image filenames.
MAE for Version 2: 8.907334345681582
-----Currently Working on Version 3-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 3: 9.23073946700496
-----Currently Working on Version 4-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 4: 8.435704722538937
-----Currently Working on Version 5-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 5: 8.544815468687016
-----Currently Working on Version 6-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 6: 9.30701340888049
-----Currently Working on Version 7-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 7: 20.582222358152656


# CHROM

**Loading the Model**

In [ ]:
json_file = open('/content/drive/MyDrive/Data/Model/VGG16/CHROM/model.json', 'r')
model_json = json_file.read()
json_file.close()
# Loading the model
model_chrom = model_from_json(model_json)
# Loading the weights
model_chrom.load_weights('/content/drive/MyDrive/Data/Model/VGG16/CHROM/model.h5')
print('Model is successfully loaded from Drive')
print(model_chrom.summary())

Model is successfully loaded from Drive
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 216, 144, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 6, 4, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________
None


In [ ]:
for i in range(1, 8):
  evaluate(model_chrom, 'Chrom', i)

-----Currently Working on Version 1-----
Total Data: 102
Found 102 validated image filenames.
MAE for Version 1: 8.543312522626067
-----Currently Working on Version 2-----
Total Data: 106
Found 106 validated image filenames.
MAE for Version 2: 8.012172448587142
-----Currently Working on Version 3-----
Total Data: 104
Found 104 validated image filenames.
MAE for Version 3: 8.31910192033826
-----Currently Working on Version 4-----
Total Data: 101
Found 101 validated image filenames.
MAE for Version 4: 7.318983203718975
-----Currently Working on Version 5-----
Total Data: 100
Found 100 validated image filenames.
MAE for Version 5: 8.68591066250225
-----Currently Working on Version 6-----
Total Data: 101
Found 101 validated image filenames.
MAE for Version 6: 8.746916500573441
-----Currently Working on Version 7-----
Total Data: 103
Found 103 validated image filenames.
MAE for Version 7: 19.55304110633073


# PCA

**Loading the Model**

In [ ]:
json_file = open('/content/drive/MyDrive/Data/Model/VGG16/PCA/model.json', 'r')
model_json = json_file.read()
json_file.close()
# Loading the model
model_pca = model_from_json(model_json)
# Loading the weights
model_pca.load_weights('/content/drive/MyDrive/Data/Model/VGG16/PCA/model.h5')
print('Model is successfully loaded from Drive')
print(model_pca.summary())

Model is successfully loaded from Drive
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 216, 144, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 6, 4, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________
None


In [ ]:
for i in range(1, 8):
  evaluate(model_pca, 'PCA', i)

-----Currently Working on Version 1-----
Total Data: 103
Found 103 validated image filenames.
MAE for Version 1: 9.29883839867496
-----Currently Working on Version 2-----
Total Data: 106
Found 106 validated image filenames.
MAE for Version 2: 9.54786481386609
-----Currently Working on Version 3-----
Total Data: 106
Found 106 validated image filenames.
MAE for Version 3: 9.458259251149071
-----Currently Working on Version 4-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 4: 8.701536117209903
-----Currently Working on Version 5-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 5: 8.940403848874125
-----Currently Working on Version 6-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 6: 9.255845217013714
-----Currently Working on Version 7-----
Total Data: 104
Found 104 validated image filenames.
MAE for Version 7: 21.884567286503707
